In [30]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from requests import get
import re
from google.colab import drive

drive.mount ('/content/drive')
dir = '/content/drive/My Drive'
json = []
i = 1
max_page = int (input ("max_page?: "))

json = []
url = "https://homes.trovit.co.za/index.php/cod.search_homes/type.1/what_d.eastern%20cape/isUserSearch.1"
response = get (url)
while i <= max_page :
    html = bs (response.content, 'html.parser')
    main_container = html.find ('ul', id ="wrapper_listing").find_all ('div', class_="snippet-content")
    for info_container in main_container :
        line = {}
        try:
            price = info_container.find ('div',class_="item-price").span.get_text()
            line['Price'] = re.sub (r'\s','',price)
        except AttributeError :
            line['Price'] = ""

        Type = info_container.parent.get ("data-property-type")
        line['Type'] = re.sub (r'\s','',Type)
        try:
          text = info_container.find ('div',class_="item-address").span.text
          content = text.split (sep=',')
          line['City'] = content[-1]
        except AttributeError :
          pass
        try:
            items = info_container.find('div',class_= "item-properties basic-properties").find_all ('div')
            for item in items :
                text = item.span.text
                content = text.split ()
                if len(content) == 2:
                  key = content[1]
                  value = content[0]
                  line[key] = value
        except AttributeError :
            pass
        json.append (line)
    i += 1

data = pd.DataFrame (json)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
max_page?: 100


In [31]:
data.rename(columns={'m²': 'Size'}, inplace=True)
data.to_csv (dir+'/trovit.csv',index = False, sep = ';')
data

,Price,Type,City,bedrooms,bathrooms,Size
0,"ZAR650,000",Homes,Eastern Cape,3,1,NaN
1,"ZAR2,200,000",Homes,Eastern Cape,3,3,NaN
2,"ZAR780,000",Homes,Eastern Cape,2,1,NaN
3,"ZAR2,200,000",Homes,Eastern Cape,2,2,NaN
4,"ZAR4,500,000",House,Eastern Cape,8,6,NaN
...,...,...,...,...,...,...
2495,"ZAR695,000",Homes,Eastern Cape,NaN,1,NaN
2496,"ZAR590,000",Homes,Gauteng,NaN,1,NaN
2497,"ZAR968,000",Flat,Eastern Cape,3,1,69
2498,"ZAR360,000",land,Eastern Cape,NaN,NaN,NaN
